# GIS Content Manager: ArcGIS Python API Automation
Automate login, content search, download, buffer, spatial join, and publishing to AGOL/Enterprise.

In [ ]:
# 1️⃣ Authentication
import json
from arcgis.gis import GIS
with open('config/credentials.json') as f:
    creds = json.load(f)
gis = GIS(creds['portal_url'], creds['username'], creds['password'])
print(f"Authenticated as: {gis.users.me.username}")

In [ ]:
# 2️⃣ Search and List Content
import pandas as pd
items = gis.content.search(query="parks AND type:Feature Layer", item_type="Feature Layer", max_items=10)
data = [
    {
        "title": item.title,
        "owner": item.owner,
        "created": item.created,
        "type": item.type,
        "id": item.id
    }
    for item in items
]
df = pd.DataFrame(data)
df.to_csv("results/portal_content_report.csv", index=False)
df

In [ ]:
# 3️⃣ Download Feature Layer Data
flayer = items[0].layers[0]
sdf = flayer.query().sdf
sdf.to_file("data/parks.geojson", driver="GeoJSON")
sdf.head()

In [ ]:
# 4️⃣ Buffer and Spatial Join (example)
import geopandas as gpd
# Example: buffer parks by 1000m
parks_buffer = sdf.copy()
parks_buffer['geometry'] = parks_buffer.geometry.buffer(1000)
parks_buffer.plot()

In [ ]:
# Spatial join demo (with another layer: e.g., crime)
# crime_sdf = gpd.read_file("data/crime.geojson")
# joined = gpd.sjoin(crime_sdf, parks_buffer, predicate='within')
# joined.to_file("results/analysis_results.geojson", driver="GeoJSON")
# joined.head()

In [ ]:
# 5️⃣ Publish and Share Results
item_properties = {
    'title': 'Buffer Analysis Results',
    'type': 'GeoJson',
    'tags': 'analysis, automation',
    'description': 'Results of buffer and spatial join analysis.'
}
result_item = gis.content.add(item_properties, data="results/analysis_results.geojson")
result_item.share(everyone=True)
result_item